In [5]:
import numpy as np
from scipy import sparse
#from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
#import nltk
#from nltk.corpus import stopwords
import string
import pandas as pd

from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_fscore_support
#nltk.download('stopwords')
#nltk.download('punkt')
#stop_words = set(stopwords.words('english') + list(string.punctuation))
#stop_words = list(stop_words)
#stop_words.sort()

In [6]:
train = pd.read_csv("data/train.csv")
content_train = train['text']
num_train = content_train.shape[0]

In [10]:
vectorizer = TfidfVectorizer(norm = 'l2', ngram_range = [1, 2])# ngram_range = [1, 2]
tf_idf_x = vectorizer.fit_transform(content_train)
print(tf_idf_x.shape)

(16000, 812229)


Select K best method

In [21]:
skbf = SelectKBest(f_classif, k = 20000)
tf_idf_x_new = skbf.fit_transform(tf_idf_x, train['label'])
print(tf_idf_x_new.shape)

(16000, 20000)


In [22]:
skbm = SelectKBest(mutual_info_classif, k = 6000)
tf_idf_x_new = skbm.fit_transform(tf_idf_x_new, train['label'])
print(tf_idf_x_new.shape)

(16000, 6000)


In [15]:
svd = TruncatedSVD(n_components = 600, n_iter=10, random_state=42)
svd.fit(tf_idf_x_new)
pc_tf_idf_x = svd.transform(tf_idf_x_new)
print(pc_tf_idf_x.shape)
print(np.sum(svd.explained_variance_ratio_))

(16000, 600)
0.5677325633908916


In [16]:
tvt = [12800, 1600, 1600]
X_train = pc_tf_idf_x[:6400, :]
y_train = train['label'][:6400]
X_test = pc_tf_idf_x[np.int(tvt[0] + tvt[1]):, :]
y_test = train['label'][np.int(tvt[0] + tvt[1]):]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(6400, 600)
(6400,)
(1600, 600)
(1600,)


In [65]:
print("HI")
# a bit of initialization
num_folds = 5
c_choice = [100, 300, 500]
kernel_choice = ['linear']
#g_choice = [1e-9, 1e-6, 1e-3, 1, 1000, 1000000]
kernel_degree = [2]
X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(y_train, num_folds)
besta= [0, 0, 0, 0]
bestf= [0, 0, 0, 0]
print("Bye")

#for g in g_choice:
for c in c_choice:
    for kc in kernel_choice:
        for kd in kernel_degree:
            averf = 0
            accuracy = 0
            for valindex in range(num_folds):
                # prepare the data
                X_traini = np.vstack((X_train_folds[0:valindex]+ X_train_folds[valindex+1:num_folds]))
                y_traini = np.hstack((y_train_folds[0:valindex]+ y_train_folds[valindex+1:num_folds]))
                X_vali = np.array(X_train_folds[valindex])
                y_vali = np.array(y_train_folds[valindex])

                # train a non-linear svm on this fold
                model = OneVsRestClassifier(SVC(C = c, kernel = kc, degree = kd, gamma = 'auto'), n_jobs = -1)
                clf = model.fit(X_traini, y_traini)
                
                accuracy += clf.score(X_vali, y_vali)
                precision, recall, f1, _ = precision_recall_fscore_support(y_train, clf.predict(X_train), average="macro")
                averf += f1

            # calculate average accuracy
            accuracy /= num_folds
            averf /= num_folds

            # memorise and update the so-far best non-linear svm, if approperiate
            if (averf > bestf[0]):
                bestf[0] = averf
                bestf[1] = c
                bestf[2] = kc
                bestf[3] = kd
                best_svmf = SVC(C = c, kernel = kc, degree = kd)
            if (accuracy > besta[0]):
                besta[0] = accuracy
                besta[1] = c
                besta[2] = kc
                besta[3] = kd
                best_svma = SVC(C = c, kernel = kc, degree = kd)

            # print the accuracy of the current non-linear svm
            print(kc, ",\tdegree: ", kd, ",\tC = ", c, ",\ntraining accuracy= ", clf.score(X_train, y_train), "\tf1: ", averf, "\tvalid accuracy= ", accuracy)

HI
Bye
linear ,	degree:  2 ,	C =  100 ,
training accuracy=  0.70890625 	f1:  0.6750776009475576 	valid accuracy=  0.5790625
linear ,	degree:  2 ,	C =  300 ,
training accuracy=  0.71921875 	f1:  0.687596591205714 	valid accuracy=  0.5721875000000001
linear ,	degree:  2 ,	C =  500 ,
training accuracy=  0.72015625 	f1:  0.6921507253361445 	valid accuracy=  0.5700000000000001


In [17]:
X_train = pc_tf_idf_x[:14400, :]
y_train = train['label'][:14400]
model = OneVsRestClassifier(SVC(C = 300, kernel = 'linear', degree = 2, gamma = 'auto'), n_jobs = -1)
clf = model.fit(X_train, y_train)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, clf.predict(X_train), average="macro")
print(clf.score(X_test, y_test), " ", f1)

0.659375   0.628413841106995


In [18]:
X_train = pc_tf_idf_x[:16000, :]
y_train = train['label'][:16000]
modelf = OneVsRestClassifier(SVC(C = 300, kernel = 'linear', degree = 2, gamma = 'auto'), n_jobs = -1)
clff = modelf.fit(X_train, y_train)

In [19]:
test = pd.read_csv("data/test.csv")
content_test = test['text']
print(content_test.shape)
tf_idf_x_test = vectorizer.transform(content_test)
print(tf_idf_x_test.shape)

(4491,)
(4491, 812229)


In [23]:
tf_idf_x_test_new = skbf.transform(tf_idf_x_test)
tf_idf_x_test_new = skbm.transform(tf_idf_x_test_new)
pc_tf_idf_test = svd.transform(tf_idf_x_test_new)
print(pc_tf_idf_test.shape)

(4491, 600)


In [24]:
print(tf_idf_x_test_new.shape)

(4491, 6000)


In [25]:
id_test = test['id']
sub_df = pd.DataFrame()
sub_df["id"] = id_test
pred = clff.predict(pc_tf_idf_test)
sub_df["pred"] = pred
sub_df.to_csv("submission.csv", index=False)